In [ ]:
import numpy as np                     
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
# Customise our plotting settings
sns.set_style('whitegrid')

#Libraries for data cleaning and preprocessing
#from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import resample
import string
import re
import pickle
import nltk

#Libraries for data preparation and model building
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score # Classification report

In [ ]:
#load the training and test data set
df_train = pd.read_csv('train_set.csv')
df_test = pd.read_csv('test_set.csv')

In [ ]:
df_train.head()

In [ ]:
df_test.head()

## Explorative Data Analysis

In [ ]:
df = df_train.copy()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df['lang_id'].value_counts()

In [ ]:
#ploting the distribution of unique label values
f, ax = plt.subplots(figsize=(10, 10))
ax = sns.countplot(x="lang_id", data=df)
plt.show()

In [ ]:
df['text'].tail(20)

In [ ]:
string.punctuation

In [ ]:
#Data preprocessing
#function that handles the removal punctuations from the tweets
def remove_punct(text):
    """
    the function remove_punction, it takes in a text as input and loops through
    the text, if a character is not in string.punctuation then it adds the character
    as a string to the text variable
    
    """
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

In [ ]:
#removes all websites and replaces them with the text 'web-url'
pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
subs_url = r'url-web'
df['message_punct'] = df['text'].replace(to_replace = pattern_url, value = subs_url, regex = True)

In [ ]:
#apply the remve_puct func to the tweets column
df['message_punct'] = df['message_punct'].apply(lambda x: remove_punct(x))
df

In [ ]:
df['message_punct'].tail(20)

## Feature Engineering

In [ ]:
#applying tokenization to the data set
tokeniser = TreebankWordTokenizer()
df['tokens'] = df['message_punct'].apply(tokeniser.tokenize)

In [ ]:
#applying Lammetization
lemmatizer = WordNetLemmatizer()
#function that handles the process of lemmatization
def extract_lemma(words, lemmatizer):
    return ' '.join([lemmatizer.lemmatize(word) for word in words]) 

In [ ]:
#calling extract_lemma function on the tokens column
df['lemma'] = df['tokens'].apply(extract_lemma, args=(lemmatizer, ))

In [ ]:
#using countVectorizer
vectorizer = CountVectorizer(max_features = 27000, analyzer = "word", ngram_range = (1,3))

In [ ]:
#transforming the data using the vectorizer
X_count = vectorizer.fit_transform(df['lemma'].values.astype(str))

In [ ]:
X_count.shape

In [ ]:
X = X_count.toarray()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#Fit label encoder and return encoded labels
y = le.fit_transform(df['lang_id'])

In [ ]:
type_labels = (le.classes_)

In [ ]:
type_labels

### Applying same for the test data

In [ ]:
test_df = df_test.copy()

In [ ]:
test_df

In [ ]:
#removing http and replacing it with url pattern
test_df['message_punct'] = test_df['text'].replace(to_replace = pattern_url, value = subs_url, regex = True)

In [ ]:
#applying the remove punctuation funtion to the test data
test_df['message_punct'] = test_df['message_punct'].apply(lambda x: remove_punct(x))
test_df.head()

In [ ]:
#applying tokenizer
test_df['tokens'] = test_df['message_punct'].apply(tokeniser.tokenize)

In [ ]:
#applying the extract_lemma function
test_df['lemma'] = test_df['tokens'].apply(extract_lemma, args=(lemmatizer, ))

In [ ]:
#transforming the data using vectorizer
test_count = vectorizer.transform(test_df['lemma'].values.astype(str))

In [ ]:
#selecting the feature
x_test = test_count.toarray()

In [ ]:
x_test.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=27)

In [ ]:
# Here 'ovr' indicates that we have selected our One-vs-Rest strategy. 
logreg = LogisticRegression(multi_class='ovr')

In [ ]:
logreg.fit(X_train, y_train)

In [ ]:
y_pred = logreg.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, target_names= type_labels))

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#making prediction on the test dataset
prediction = logreg.predict(x_test)

In [ ]:
#creating a dataframe for the submission
submission = pd.DataFrame(list(zip(test_df['index'],  le.inverse_transform(prediction))), columns = ['index', 'lang_id'])
submission.head()

In [ ]:
#saving the file as csv
submission.to_csv('submission_GCE1.csv', index_label = False, index = False)

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(30, 30, 30), max_iter=100000)  
mlp.fit(X_train, y_train)

In [ ]:
y_mlp = mlp.predict(X_test)

In [ ]:
print(classification_report(y_test, y_mlp, target_names= type_labels))

import warnings
warnings.filterwarnings('ignore')

In [ ]:
predictions7 = mlp.predict(x_test)

In [ ]:
#creating a dataframe for the submission
submission7 = pd.DataFrame(list(zip(test_df['index'],  le.inverse_transform(predictions7))), columns = ['index', 'lang_id'])
submission7.head()

In [ ]:
submission7.to_csv('submission_GCE8.csv', index_label = False, index = False)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
nb = GaussianNB()
# Fit the model 
nb.fit(X_train, y_train)

In [ ]:
y_nb = nb.predict(X_test)

In [ ]:
print(classification_report(y_test, y_nb, target_names= type_labels))

import warnings
warnings.filterwarnings('ignore')

In [ ]:
predictions5 = mlp.predict(x_test)

In [ ]:
#creating a dataframe for the submission
submission5 = pd.DataFrame(list(zip(test_df['index'],  le.inverse_transform(predictions5))), columns = ['index', 'lang_id'])
submission5.head()

In [ ]:
submission5.to_csv('submission_GCE7.csv', index_label = False, index = False)